In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/Technical-Writer-Assistant')
! pwd

/content/drive/My Drive/Documents/Imperial/Individual_project/Technical-Writer-Assistant


In [3]:
# Load a sample from .txt data file
import explore_data
from importlib import reload
reload(explore_data)

da = explore_data.DatasetExplorer('pubmed', 'colab')
samples = da.fetch_samples()
sample = samples[0]['article_text']
print(samples[0].keys())
print(sample)

# Tokenize and truncate so can fit within Longformer
flat_sample = [item for list in sample for item in list.split()]
x = ' '.join(flat_sample[:1000])
print(x)

dict_keys(['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections'])
['approximately , one - third of patients with symptomatic vte manifests pe , whereas two - thirds manifest dvt alone .', 'both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected .', 'if undiagnosed , asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension .', 'vte is not only disabling but also prolongs hospital stay and increases the cost of treatment .', 'along with myocardial infarction and arrhythmia ( due to electrolyte imbalance ) , pe is one of the commonest causes of sudden unexplained deaths in hospitalized patients .', 'it is estimated that 20 million cases of lower extremity dvt occur in the usa alone .', 'the prevailing notion that the incidence of vte in asians is less than that in the western population has been dispro

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
os.chdir('/content/drive/My Drive/Documents/Imperial/Individual_project/longformer')
! pwd
# ! pip install -r requirements.txt

/content/drive/My Drive/Documents/Imperial/Individual_project/longformer


In [6]:
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer

config = LongformerConfig.from_pretrained('longformer-base-4096/')
# choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
# 'n2': for regular n2 attantion
# 'tvm': a custom CUDA kernel implementation of our sliding window attention
# 'sliding_chunks': a PyTorch implementation of our sliding window attention
config.attention_mode = 'tvm'

model = Longformer.from_pretrained('longformer-base-4096/', config=config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.model_max_length = model.config.max_position_embeddings

# SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document
# SAMPLE_TEXT = f'{tokenizer.cls_token}{SAMPLE_TEXT}{tokenizer.eos_token}'
SAMPLE_TEXT = f'{tokenizer.cls_token}{x}{tokenizer.eos_token}'

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1
input_ids = input_ids.repeat(1, 1)

# TVM code doesn't work on CPU. Uncomment this if `config.attention_mode = 'tvm'`
model = model.cuda(); input_ids = input_ids.cuda()

# Attention mask values -- 0: no attention, 1: local attention, 2: global attention
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens

# padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
input_ids, attention_mask = pad_to_window_size(
        input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

print(input_ids.shape)
output = model(input_ids, attention_mask=attention_mask)[0]

print('> ', output.shape)

torch.Size([1, 1536])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
torch.Size([1, 1536, 768])
>  torch.Size([1, 1536, 768])


In [0]:
# del model, output
del attention_mask
torch.cuda.empty_cache()